## Configuração inicial

In [1]:
import pickle
import pandas as pd

from Utils import templates, utils, recommender, models_config

In [2]:
## Configurações base

config = {
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_1m",                                                  #| Opções 'ml_100k' e 'ml_1m'
    "nsu" : 12,     # número de usuários para filtragem colaborativa        | SSBD :12  | Default :18   |            | Best = 19
    "nci" :19,      # número de itens para filtragem colaborativa           | SSBD :19  | Default :24   | Max : 1682 |
    "lenlimit" : 8,  # limite de tamanho para a lista filmes assistidos     | SSBD : 8  | Default :24   | Max : 1682 | Best = 8
    "lenlimit_option" : 'ultimos', # define qual a abordagem                | opções : 'ultimos', 'primeiros', 'aleatorio' | Default : 'ultimos'
    "test_run" : 0, # define a quantidade de recomendações,                 |           | Default :0    | Max : 943 
    "obs": "testando dataset"
}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_3
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json(f"Data/{config['dataset']}.json")
print(f'Quantidade de Usuários: {len(dataset)}')

Quantidade de Usuários: 6040


In [3]:
config.update(models_config.qwen_2_5_7b_instruct_1m())

## Execução unitária

In [ ]:
result_pkl = recommender.recommendation_workflow(config         = config,
                                                 dataset        = dataset,
                                                 prompt_template= prompt_template,
                                                 prompt_format  = prompt_format)

## Execução multipla 

In [6]:
for i in range(0, len(config_list)):
    config = config_list[i]
    print(config)

{'LLM_runtime': 'Vulkan llama.cpp v1.23.0', 'dataset': 'ml_100k', 'nsu': 12, 'nci': 19, 'lenlimit': 8, 'lenlimit_option': 'ultimos', 'test_run': 0, 'obs': 'estando dataset', 'prompt_template': {'System_prompt': "You are a movie expert provide the answer for the question based on the given context. If you don't know the answer to a question, please don't share false information.", 'Preference': '\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ### QUESTION: Based on my watched movies list. Tell me what features are most important to me when selecting movies (Summarize my preferences briefly)?\n\n    ### ANSWER:\n    ', 'Featured_movies': '\n\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ### MY MOVIE PREFERENCES: {}.\n\n    ### QUESTION: Create an enumerated list selecting the five most featured movies from the watched movies according to my movie preferences.\n\n    ### ANSWER:\n    ', 'Recommendation': '\n\n    ### CANDIDATE MOVIE SET: 

In [7]:
config_list = []

config.update({"dataset": "ml_100k"})
config.update({"obs": "estando dataset"})
config.update(models_config.qwen_2_5_7b_instruct_1m())

config1 = config.copy()
config_list.append(config1)

config2 = config1.copy()
config_list.append(config2)

config3 = config.copy()
config3.update({"dataset": "ml_1m"})
config3.update(models_config.gemma_3_4b_it())
config_list.append(config3)

config4 = config3.copy()
config_list.append(config4)

config5 = config.copy()
config5.update({"dataset": "ml_100k"})
config5.update(models_config.gemma_3_4b_it())
config_list.append(config5)

config6 = config5.copy()
config_list.append(config6)

for i in range(0, len(config_list)):
    config = config_list[i]
    dataset = utils.read_json(f"Data/{config['dataset']}.json")
    print(f'Rodando configuração {i} de {len(config_list)}')
    try:
        result_pkl = recommender.recommendation_workflow(config         = config,
                                                         dataset        = dataset,
                                                        prompt_template= prompt_template,
                                                        prompt_format  = prompt_format)
        print(f'Configuração {i} de {len(config_list)} finalizada')
    except Exception as e:
        print(f'Erro na configuração {i} de {len(config_list)}')
        print(e)
        continue


Rodando configuração 0 de 6


Processando:   0%|          | 0/943 [00:00<?, ?it/s]

Configuração 0 de 6 finalizada
Rodando configuração 1 de 6


Processando:   0%|          | 0/943 [00:00<?, ?it/s]

Configuração 1 de 6 finalizada
Rodando configuração 2 de 6


...

Configuração 3 de 6 finalizada
Rodando configuração 4 de 6


Processando:   0%|          | 0/943 [00:00<?, ?it/s]

Configuração 4 de 6 finalizada
Rodando configuração 5 de 6


Processando:   0%|          | 0/943 [00:00<?, ?it/s]

Configuração 5 de 6 finalizada


# outros

In [ ]:
# retorna apenas o usuários que tem o gt no candidatos.
# implementar sobre o pipeline do projeto 

def get_candidate_ids_list(data, id_list, user_matrix_sim, num_u, num_i):
    cand_ids = []
    for i in id_list:
        watched_movies = data[i][0].split(' | ')
        candidate_items = utils.sort_user_filtering_items(data, watched_movies, user_matrix_sim[i], num_u, num_i)
        if data[i][-1] in candidate_items:
            cand_ids.append(i)
    return cand_ids

In [ ]:
id_list = list(range(0, len(dataset)))
#assert(len(id_list) == 943) # aqui é verificado se a lista possue exatamente essa quantidade

# Building indexes and similarity matrices for users and movies.
movie_idx = utils.build_moviename_index_dict(dataset)
user_sim_matrix = utils.build_user_similarity_matrix(dataset, movie_idx)


# para fazer a filtragem sobre os filmes 
#pop_dict = utils.build_movie_popularity_dict(dataset) 
#item_sim_matrix = utils.build_item_similarity_matrix(dataset)

## Execução

In [ ]:
result_pkl = recommender.recommendation_workflow(config         = config,
                                                 dataset        = dataset,
                                                 prompt_template= prompt_template,
                                                 prompt_format  = prompt_format)

## execução varias configs

In [ ]:
import pickle
import pandas as pd

from Utils import templates, utils, recommender

## PROMPT 2 

config = {}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_2
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

config.update({
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 19,     # número de usuários para filtragem colaborativa        | SSBD :12  | Default :18   | 
    "nci" :19,      # número de itens para filtragem colaborativa           | SSBD :19  | Default :24   | Max : 1682
    "lenlimit" : 8,  # limite de tamanho para a lista filmes assistidos    | SSBD : 8  | Default :24   | Max : 1682
    "test_run" : 0, # define a quantidade de recomendações,                 |           | Default :0    | Max : 943 
    "obs": "testando prompt novo - prompt2"
})

config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

config_list = []

config1 = config.copy()
config_list.append(config1)

for i in range(0, len(config_list)):
    config = config_list[i]
    print(f'Rodando configuração {i} de {len(config_list)}')
    try:
        result_pkl = recommender.recommendation_workflow(config         = config,
                                                         dataset        = dataset,
                                                        prompt_template= prompt_template,
                                                        prompt_format  = prompt_format)
        print(f'Configuração {i} de {len(config_list)} finalizada')
    except Exception as e:
        print(f'Erro na configuração {i} de {len(config_list)}')
        print(e)
        continue

## PROMPT 3 

config = {}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_3
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

config.update({
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 12,     # número de usuários para filtragem colaborativa        | SSBD :12  | Default :18   | 
    "nci" :19,      # número de itens para filtragem colaborativa           | SSBD :19  | Default :24   | Max : 1682
    "lenlimit" : 8,  # limite de tamanho para a lista filmes assistidos    | SSBD : 8  | Default :24   | Max : 1682
    "test_run" : 0, # define a quantidade de recomendações,                 |           | Default :0    | Max : 943 
    "obs": "testando prompt novo - prompt3"
})

config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

config_list = []

config1 = config.copy()
config_list.append(config1)

for i in range(0, len(config_list)):
    config = config_list[i]
    print(f'Rodando configuração {i} de {len(config_list)}')
    try:
        result_pkl = recommender.recommendation_workflow(config         = config,
                                                         dataset        = dataset,
                                                        prompt_template= prompt_template,
                                                        prompt_format  = prompt_format)
        print(f'Configuração {i} de {len(config_list)} finalizada')
    except Exception as e:
        print(f'Erro na configuração {i} de {len(config_list)}')
        print(e)
        continue

In [ ]:
for i in range(0, len(config_list)):
    config = config_list[i]
    print(config)

## Resultados

In [ ]:
with open(f'{result_pkl}', 'rb') as f:
    data = pickle.load(f)

In [ ]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
            #'Recommendations': value.get('recommendations', ''),
            'rec_HitRate@10': value.get('rec_HitRate@10', ''),
            #'Precision': value.get('precision', ''),
            #'Recall': value.get('recall', ''),
            'rec_NDCG@10': value.get('rec_NDCG@10', '')
        })

df_results = pd.DataFrame(results)

df_results

# Preparar dataset / dividir em treino e teste

## Cria uma lista dos usuários com GT na lista de candidatos

In [ ]:
import random
import pandas as pd
from Utils import templates, utils

nsu = 12
nci = 19
lenlimit = 8 

dataset = utils.read_json("Data/ML100K_clean.json")
movie_idx = utils.build_moviename_index_dict(dataset)
user_sim_matrix = utils.build_user_similarity_matrix(dataset, movie_idx)
id_list = list(range(0, len(dataset)))

data_list = []
id_list_com_gt_no_candidate=[]

for i in id_list:

  watched_mv = dataset[i][0].split(' | ')[::-1]
  watched_mv = watched_mv[-lenlimit:]
  
  groundTruth = dataset[i][-1]
  candidate_items = utils.sort_collaborative_user_filtering(target_user_id=i,
                                                                    dataset=dataset,
                                                                    user_similarity_matrix=user_sim_matrix,
                                                                    num_users=nsu,
                                                                    num_items=nci,
                                                                    include_similar_user_GT=False,
                                                                    debug=False)
  random.shuffle(candidate_items)

  # verifica se o ground_truth está no candidate_set
  gt_in_candidate_set = True if any(groundTruth.lower() in candidate.lower() for candidate in candidate_items) else False

  if gt_in_candidate_set == True:
    id_list_com_gt_no_candidate.append(i)

  data_list.append({
        'user_id': i,
        'watched_movies': watched_mv,
        'ground_truth': groundTruth,
        'candidate_items': candidate_items,
        'gt_in_candidate_set': gt_in_candidate_set
  })

df = pd.DataFrame(data_list)

print(f'nsu: {nsu} \nnci: {nci} \nQnt de usuários do dataset: {len(dataset)} \nQnt de usuários com gt no candidate: {len(id_list_com_gt_no_candidate)}')

## Realiza a divisão de treino e teste

In [ ]:
# Define a proporção de divisão
train_ratio = 0.8
test_ratio = 0.2

# Embaralha os IDs para garantir aleatoriedade
random.shuffle(id_list_com_gt_no_candidate)

# Calcula o tamanho de cada conjunto
total_ids = len(id_list_com_gt_no_candidate)
train_size = int(train_ratio * total_ids)
test_size = total_ids - train_size

# Separa os IDs em conjuntos
train_ids = id_list_com_gt_no_candidate[:train_size]
test_ids = id_list_com_gt_no_candidate[train_size:]

# Imprime o tamanho de cada conjunto para verificação
print(f"Tamanho do conjunto de treino: {len(train_ids)}")
print(f"Tamanho do conjunto de teste: {len(test_ids)}")

## Ajusta o dataset de treino, adicionando as instruções necessárias

In [ ]:
train_dataset = []

# configs 
model_name = "gemma-3-4b-it"
temperature = 0
prompt_template = templates.PROMPT_TEMPLATE_3
system_prompt = "You are a movie expert provide the answer for the question based on the given context. Your answer must be short " 
max_tokens = -1
lenlimit = 8

for id in train_ids:
    filtered = df[df['user_id'] == id]
    if not filtered.empty:
        candidate_items = filtered['candidate_items'].iloc[0]
        ground_truth = filtered['ground_truth'].iloc[0]
        watched_movies = filtered['watched_movies'].iloc[0]
    else:
        continue 

    input_prompt = prompt_template['Preference'].format(', '.join(candidate_items),', '.join(watched_movies))
    response = utils.query_lm_studio(model_name,0.0,system_prompt,input_prompt,max_tokens)
    movie_preference = response

    input_prompt = prompt_template['Featured_movies'].format(', '.join(watched_mv), movie_preference)
    response = utils.query_lm_studio(model_name,0.0,system_prompt,input_prompt,max_tokens)
    most_featured = response

    input = templates.TRAIN_TEMPLATE_2['INPUT'].format(', '.join(candidate_items),', '.join(watched_movies),movie_preference,most_featured )
    output= templates.TRAIN_TEMPLATE_2['OUTPUT'].format(ground_truth)

    train_dataset.append({
        'input': input,
        'output': output
  })



In [ ]:
df_train = pd.DataFrame(train_dataset)
df_train



## Salva arquivo de treino

In [ ]:
df_train.to_pickle('Data/ML100K_train.pkl')

## Salva arquivo de teste

In [ ]:
test_dataset = []

for id in test_ids:
    filtered = df[df['user_id'] == id]
    if not filtered.empty:
        candidate_items = filtered['candidate_items'].iloc[0]
        ground_truth = filtered['ground_truth'].iloc[0]
        watched_movies = filtered['watched_movies'].iloc[0]
    else:
        continue 
    
    test_dataset.append({
        'user_id': id,
        'candidate_items': candidate_items,
        'ground_truth': ground_truth,
        'watched_movies': watched_movies
    })
df_test = pd.DataFrame(test_dataset)
df_test.to_pickle('Data/ML100K_test.pkl')
df_test